IN Memory Vector Store

In [1]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

embeddings=OpenAIEmbeddings(model='text-embedding-3-small')

vector_store=InMemoryVectorStore(embedding=embeddings)

docs=[
    Document(page_content='Ram am a boy'),
    Document(page_content='Ram lives in India'),
    Document(page_content='Ram wants to work in USA')
]

vector_store.add_documents(docs)

results=vector_store.similarity_search("Where ram lives ?",k=1)

print(results)




[Document(id='09217019-238c-44e0-aad0-d628bedf5447', metadata={}, page_content='Ram lives in India')]


FAISS Vector Store

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from pathlib import Path

pdf_path=list(Path('SRCs').glob('*.pdf'))[0]
loader=PyPDFLoader(pdf_path)
docs=loader.load()

splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=[
        "\n\n", "\n", " ", ""
    ]
)

embeddings=OpenAIEmbeddings(
    model='text-embedding-3-small'
)

splitted_text=splitter.split_documents(docs)
vector_store=FAISS.from_documents(splitted_text,embedding=embeddings)

vector_store.save_local('vector_store')

result=vector_store.similarity_search('what is the ancient relation of cow and humans',k=3)
print(result)



[Document(id='3eeb74de-5ff8-4512-b7fe-90d1938c4fb6', metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2026-01-10T15:00:21+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2026-01-10T15:00:21+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'SRCs/cow_paragraph.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='thousands of years. From ancient times, cows have been closely associated with human survival,\nproviding essential resources such as milk, which is a primary source of nutrition containing\nproteins, calcium, vitamins, and healthy fats that support growth and overall well-being. Beyond\nmilk, cows contribute to the production of dairy products like curd, butter, cheese, and ghee, all of\nwhich form an important part of diets around the world. In rural economies, especially in agrarian\nsocieties, cows are often considered a backbone of livelihood,

Loading Local FAISS Index

In [8]:
from pathlib import Path

faiss_index_path=Path('vector_store')

if faiss_index_path.exists():
    load_stores=FAISS.load_local(
        folder_path='vector_store',
        embeddings=embeddings,
        allow_dangerous_deserialization=True
    )

result=load_stores.similarity_search('what is the ancient relation of cow and humans',k=3)

print(result[0].page_content)




thousands of years. From ancient times, cows have been closely associated with human survival,
providing essential resources such as milk, which is a primary source of nutrition containing
proteins, calcium, vitamins, and healthy fats that support growth and overall well-being. Beyond
milk, cows contribute to the production of dairy products like curd, butter, cheese, and ghee, all of
which form an important part of diets around the world. In rural economies, especially in agrarian
societies, cows are often considered a backbone of livelihood, as they support farmers not only
through dairy production but also through organic manure, which enriches soil fertility and promotes
sustainable farming practices. The calm and non-aggressive nature of cows makes them easy to
domesticate, and their presence on farms often symbolizes stability, patience, and harmony
between humans and nature. Cows are also deeply embedded in cultural and religious traditions,


Chroma Database

In [9]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document


embeddings=OpenAIEmbeddings(model='text-embedding-3-small')

vector_store=Chroma(
    collection_name='cow_details',
    embedding_function=embeddings,
    persist_directory='vector_store'
)

sample_docs=[Document(
    page_content='Rag reduces the Cost of Fine Tuning',
    metadata={'topic':'Rag','difficulty':'Easy'}
),
Document(
    page_content='Langchain used Vector Embeding',
    metadata={'topic':'LangChain','difficulty':'Intermediate'}
)
]

vector_store.add_documents(sample_docs)

result=vector_store.similarity_search(
    "How can we reduce the Cost of Fine Tuning",k=2,filter={'topic':'Rag'}
)

print(result[0].page_content)



Rag reduces the Cost of Fine Tuning
